In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import jieba.posseg
import jieba.analyse
import pandas as pd

#### 1. 抽取特定词性的词

In [2]:
def dataPrepos(text):
    l = []
    pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd']  # 定义选取的词性
    seg = jieba.posseg.cut(text)  # 分词
    for i in seg:
        if i.word and i.flag in pos:  # 词性筛选
            l.append(i.word)
    return l

In [3]:
data = pd.read_csv('data/sample_data.csv')

In [4]:
data.head()

,id,title,abstract
0,1,永磁电机驱动的纯电动大巴车坡道起步防溜策略,本发明公开了一种永磁电机驱动的纯电动大巴车坡道起步防溜策略，即本策略当制动踏板已踩下、永磁电...
1,2,机动车辆车门的肘靠,一种溃缩结构是作为内部支撑件而被提供在机动车辆的车门衬板上的肘靠中，所述溃缩结构具有多个以交...
2,3,仪表板支撑结构,本发明公开了一种支撑结构，其配置用于在车辆的乘客车厢内定位仪表板。所述支撑结构包括支撑支架和...
3,4,铰接的头枕总成,一种车辆座椅总成，包括座椅靠背、头枕和支承结构，支承结构在头枕和座椅靠背之间延伸。支承结构包...
4,5,用于评估和控制电池系统的系统和方法,本发明涉及用于评估和控制电池系统的系统和方法。介绍了用于估计电池系统中的各独立电池分部的相对...


In [5]:
def getKeywords_tfidf(data,topK):
    idList, titleList, abstractList = data['id'], data['title'], data['abstract']
    corpus = [] # 将所有文档输出到一个list中，一行就是一个文档
    for index in range(len(idList)):
        text = '%s。%s' % (titleList[index], abstractList[index]) # 拼接标题和摘要
        text = dataPrepos(text) # 文本预处理
        text = " ".join(text) # 连接成字符串，空格分隔
        corpus.append(text)
    return corpus

In [6]:
corpus = getKeywords_tfidf(data=data,topK=10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.861 seconds.
Prefix dict has been built succesfully.


#### 2. 每一篇文章基于tf-idf 抽取Top 10 的关键词 用 ‘ ’ 分隔

In [8]:
corpus[:3] 

['永磁 电机 驱动 纯 电动 大巴车 坡道 起步 防溜 策略 本发明 永磁 电机 驱动 纯 电动 大巴车 坡道 起步 防溜 策略 即 策略 制动 踏板 已 踩下 永磁 电机 转速 小于 设定值 持续 一定 时间 整车 控制 单元 产生 刹车 触发 信号 油门 踏板 开度 小于 设定值 档位 装置 时 电机 控制 单元 产生 防溜 功能 使能 信号 自动 进入 防溜 控制 使 永磁 电机 进入 转速 闭环控制 目标 转速 整车 控制 单元 检测 到 制动 踏板 仍然 踩下 则 限制 永磁 电机 输出 力矩 恢复 永磁 电机 输出 力矩 整车 控制 单元 检测 到 油门 踏板 开度 大于 设置 值 档位 装置 手刹 装置 处于 驻车 位置 则 退出 防溜 控制 切换 到 力矩 控制 策略 无需 更改 车辆 结构 添加 辅助 传感器 硬件 设备 实现 车辆 防溜 目的',
 '机动车辆 车门 靠 溃缩 结构 是 作为 支撑 提供 机动车辆 车门 衬板 靠 溃缩 结构 具有 交叉 形 方式 设计 凹陷 装饰 覆盖 泡沫 元件 安排 溃缩 结构 溃缩 结构 特别 用于 吸收 碰撞 事件 负荷 防止 车辆 乘车 免受 增加 力 峰值',
 '仪表板 支撑 结构 本发明 支撑 结构 配置 用于 车辆 乘客 车厢 内 定位 仪表板 支撑 结构 包括 支撑 支架 端部 支架 支撑 支架 附 接到 车辆 车身 面板 支撑 支架 包括 横向 偏压 导引 端部 支架 附 接到 仪表板 端部 支架 具有 邻接 支撑 支架 横向 偏压 导引 使得 横向 偏压 导引 横向 方向 偏压 仪表板 直到 端部 支架 邻接 车身 面板 横向 基准 表面']

In [9]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [10]:
corpus = [i.split(' ') for i in corpus]

- window 词窗口 左边 5 个词 右边 5 个词
- min_count 词频最低要求
- worker 并行计算个数
- sg: Skip-Gram  = 1， CBOW =0 
- negative: 负采样的个数， 要求多少个负采样的个数分数为底
- n_exponenet = 0.75

In [13]:
model = Word2Vec(corpus, size=100, window=5, min_count=1, workers=4, sg = 1, negative = 5, ns_exponent = 0.75)

###### 通过训练模型后 输出这个词最相近的词

In [15]:
model.most_similar('横向')

C:\Users\DELL\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('具有', 0.3485231399536133),
 ('运行', 0.34649547934532166),
 ('表面', 0.32488691806793213),
 ('过程', 0.3196510970592499),
 ('包括', 0.2992086112499237),
 ('结构', 0.2987203896045685),
 ('电池', 0.2976333498954773),
 ('满足', 0.29630720615386963),
 ('单元', 0.2953327000141144),
 ('头枕', 0.29290771484375)]

In [18]:
model.vocabulary